## 1.3 노드와 에지의 피처
노드들과 에지들의 그래프별 속성을 저장하기 위해서, `DGLGraph` 의 노드들과 에지들은 이름을 갖는 사용자 정의 피쳐를 갖을 수 있다. `ndata` 와 `edata` 인터페이스를 이용해서 이 피쳐들을 접근할 수 있다. 
#### 1. 두 노드에 대한 피쳐를 생성하고(라인 8과 15에서 'x' 와 'y' 이름 피처), 한개의 에지 피처(라인 9에서 'x' 이름 피처)를 생성

In [3]:
import dgl
import torch as th
g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0])) # 6 nodes, 4 edges
g
g.ndata['x'] = th.ones(g.num_nodes(), 3)               # node feature of length 3
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # scalar integer feature
g
# different names can have different shapes
g.ndata['y'] = th.randn(g.num_nodes(), 5)
g.ndata['x'][1]                  # get node 1's feature
g.edata['x'][th.tensor([0, 3])]  # get features of edge 0 and 3

tensor([1, 1], dtype=torch.int32)

- `ndata`/`edata` 인터페이스의 중요한 사실들:
    - 숫자 타입(예, float, double, int)의 피처들만 허용된다. 피처는 스칼라, 벡터, 또는 다차원 텐서가 가능하다.
    - 각 노드 피처는 고유한 이름을 갖고, 각 에지 피쳐도 고유한 이름을 갖는다. 노드와 에지의 피쳐는 같은 이름을 갖을 수 있다. (예, 위 예의 ‘x’)
    - 텐서 할당으로 피처가 만들어진다. 즉, 피처를 그래프의 각 노드/에지에 할당하는 것이다. 텐서의 첫번째 차원은 그래프의 노드/에지들의 개수와 같아야 한다. 그래프의 노드/에지의 일부에만 피쳐를 할당하는 것은 불가능하다.
    - 같은 이름의 피처들은 같은 차원 및 같은 타입을 갖아야 한다.
    - 피처 텐서는 행 위주(row-major)의 레이아웃을 따른다. 각 행-슬라이스는 한 노드 또는 이제의 피처를 저장한다. (아래 예제의 16줄 및 18줄을 보자)

#### 2. 가중치 그래프인 경우, 에지 피처로 가중치를 저장할 수 있다.

In [4]:
# edges 0->1, 0->2, 0->3, 1->3
edges = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
weights = th.tensor([0.1, 0.6, 0.9, 0.7])  # weight of each edge
g = dgl.graph(edges)
g.edata['w'] = weights  # give it a name 'w'
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

## 1.4 외부 소스를 사용한 그래프 생성하기
- 외부 소스들로부터 `DGLGraph`를 만드는 옵션들:
    - 그래프 및 회소 행렬을 위한 python 라이브러리(NetworkX 및 SciPy)로부터 변환하기
    - 디스크에서 그래프를 로딩하기
\
이 절에서는 다른 그래프를 변환해서 그래프를 생성하는 함수들은 다루지 않겠다. 그 방법들에 대한 소개는 매뉴얼의 API를 참조하자.
#### 1. 외부 라이브러리를 사용해서 그래프 생성하기
- 아래 코드는 SciPy 희소행렬과 NetworkX 그래프로부터 그래프를 생성하는 예제

In [5]:
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5% nonzero entries
dgl.from_scipy(spmat)                   # from SciPy
import networkx as nx
nx_g = nx.path_graph(5) # a chain 0-1-2-3-4
dgl.from_networkx(nx_g) # from networkx

Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

`nx.path_graph(5)` 로부터 만들면 생성된 `DGLGraph`는 4개가 아니라 8개의 에지를 갖는 점을 유의하자. 이유는 `nx.path_graph(5)`는 방향이 없는 NetworkX 그래프 `networkx.Graph`를 만드는데, `DGLGraph`는 항상 방향이 있는 그래프이기 때문이다. 방향이 없는 NetworkX 그래프를 `DGLGraph`로 변환하면, DGL은 내부적으로 방향이 없는 에지를 두개의 방향이 있는 에지로 변환한다. `networkx.DiGraph`를 사용하면 이런 현상을 피할 수 있다.

In [6]:
nxg = nx.DiGraph([(2, 1), (1, 2), (2, 3), (0, 0)])
dgl.from_networkx(nxg)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

- NOTE : 내부적으로 DGL은 SciPy 행렬과 NetworkX 그래프를 텐서로 변환해서 그래프를 만든다. 따라서, 이 생성 방법은 성능이 중요한 곳에 사용되기 적합하지 않다.
#### 2. 디스크에서 그래프 로딩하기
##### (1) Comma Separated Values (CSV)
- https://github.com/dglai/WWW20-Hands-on-Tutorial/blob/master/basic_tasks/1_load_data.ipynb
##### (2) JSON/GML 포멧
##### (3) DGL 바이너리 포멧
